In [ ]:
import numpy as np
import anndata
import sys  
sys.path.insert(1, '../../backend/')
from dataset import Dataset as TraceData
from tsne import compute_tsne_series

In [ ]:
adata = anndata.read_h5ad("./fly_atlas_tsne.h5ad", 'r')

In [ ]:
adata.uns['methods']

In [ ]:
tsne_embeddings = compute_tsne_series(
    data=np.asarray(adata.X),
    init=adata.obsm["X_pca"][:, 0:2],
    sampling_frac=0.01,
    coarse_exag_iter=[(12, 200), (10, 200)],
    fine_exag_iter=[(10, 200), (5, 200), (3, 200), (1, 200)],
    smoothing_perplexity=30,
    random_state=42,
    hd_metric="euclidean",
    negative_gradient_method="fft",
    fpath_prefix="fly_atlas"
)

In [ ]:
from importlib import reload
import dataset
reload(dataset)
from dataset import Dataset as TraceData

In [ ]:
trace_data = TraceData(
    adata=adata,
    name="Fly",
    hd_metric="euclidean",
    hd_data_key="X_pca",
    verbose=True
)

In [ ]:
for exag, emb in tsne_embeddings.items():
    trace_data.add_embedding(
        emb,
        name=f"tSNE (exag {exag})",
        category="tSNE",
        meta_info={"params": {"exaggeration": exag, "hd_data": "PCA 200"}}
    )

In [ ]:
trace_data.compute_neighborhood_preservation(
    neighborhood_sizes=[200, 100, 50]
)
trace_data.compute_global_distance_correlation(
    max_landmarks=1000, LD_landmark_neighbors=True
)

In [ ]:
trace_data.print_quality()
trace_data.align_embeddings(reference_embedding="HVG PCA")
trace_data.save_adata(filename="./fly_atlas_tsne.h5ad")